In [1]:
import numpy as np
import matplotlib.pyplot as plt

# build a gaussian process model on the training data
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF



In [2]:
# make a simple 1D synthetic dataset for regression with only 5 data points
# the data is generated from a linear model with some noise
np.random.seed(7)
X_tr =  np.random.randn(20, 1)*15
# sin function
y_tr = 0.1* X_tr + np.sin(X_tr) + np.random.randn(20, 1)*0.01






# plot the data
plt.plot(X_tr, y_tr, "b.", label="Training data")
plt.xlabel("x")
plt.ylabel("y", rotation=0)
plt.axis([-50, 40, -4, 6])
plt.tight_layout()
plt.savefig(f"fig/example_train_points.pdf")
plt.close()




In [3]:


kernel =  RBF(length_scale=10000000) 
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=0.01)
gp.fit(X_tr, y_tr);






In [7]:
# Make a plot of the data and the model  

X = np.linspace(-50, 40, 10000).reshape(-1, 1)
y_pred, sigma = gp.predict(X, return_std=True)

plt.plot(X_tr, y_tr, "b.", label="Training data")
plt.plot(X, y_pred, "r-", label="GP prediction")
plt.fill_between(X.ravel(), y_pred.ravel() - 1.96 * sigma, y_pred.ravel() + 1.96 * sigma, alpha=0.2, color="red", label="95% CI")
plt.xlabel("x")
plt.ylabel("y", rotation=0)
plt.legend()
plt.axis([-50, 40, -4, 6])
plt.tight_layout()
plt.savefig(f"fig/example_gp_model.pdf")
plt.close()



In [5]:
##############################################################################
# Append this code below the code you have already provided
##############################################################################

# Define the x_value we want to focus on
x_value = -20

# 1) Same figure with a vertical line at x = x_value
#plt.figure(figsize=(6, 4))
plt.plot(X_tr, y_tr, "b.", label="Training data")
plt.plot(X, y_pred, "r-", label="GP prediction")
plt.fill_between(
    X.ravel(),
    (y_pred - 1.96 * sigma).ravel(),
    (y_pred + 1.96 * sigma).ravel(),
    alpha=0.2,
    color="red",
    label="95% CI",
)
# Draw vertical line at x = x_value
plt.axvline(x=x_value, color="black", linestyle="--", label=f"x = {x_value}")

plt.xlabel("x")
plt.ylabel("y", rotation=0)
plt.legend()
plt.axis([-50, 40, -4, 6])
plt.tight_layout()
plt.savefig(f"fig/example_gp_model_x_.pdf")
plt.close()





# 2) Univariate Gaussian shape at x = x_value
#    Obtain the mean and standard deviation of the GP at that point
x_point = np.array([[x_value]])  # shape (1,1)
y_point_mean, y_point_std = gp.predict(x_point, return_std=True)
y_point_mean = y_point_mean[0]  # scalar
y_point_std = y_point_std[0]    # scalar

# Build a y-range around the mean for plotting
y_min = y_point_mean - 3 * y_point_std
y_max = y_point_mean + 3 * y_point_std
y_range = np.linspace(y_min, y_max, 300)

# Compute the pdf of Normal(mean, std^2)
pdf_y = (
    1.0 / (np.sqrt(2.0 * np.pi) * y_point_std)
    * np.exp(-0.5 * ((y_range - y_point_mean) / y_point_std) ** 2)
)

#plt.figure(figsize=(6, 4))
plt.plot(y_range, pdf_y, "b-", label=f"GP marginal at x = {x_value}")
plt.xlabel("y")
plt.ylabel("pdf", rotation=0)
plt.title(
    f"Distribution of y at x={x_value}\n"
    f"(mean={y_point_mean:.3f}, std={y_point_std:.3f})"
)
plt.legend()
plt.tight_layout()
plt.savefig(f"fig/example_gp_model_x_univariate.pdf")
plt.close()


In [9]:


# sample from the GP model (after fit) 5 functions and plot them
X = np.linspace(-50, 40, 2000).reshape(-1, 1)
y_samples = gp.sample_y(X, 5)
y_pred, sigma = gp.predict(X, return_std=True)

plt.plot(X_tr, y_tr, "b.", label="Training data")
plt.plot(X, y_samples[:, 0], "g-", alpha=0.5, label="GP samples")  # Plot the first sample with label
for i in range(1, y_samples.shape[1]):
    plt.plot(X, y_samples[:, i], "g-", alpha=0.5)  # Plot the rest without labels
plt.fill_between(X.ravel(), y_pred.ravel() - 1.96 * sigma, y_pred.ravel() + 1.96 * sigma, alpha=0.1, color="red")
plt.xlabel("x")
plt.ylabel("y", rotation=0)
plt.legend() 
plt.axis([-50, 40, -4, 6])
plt.tight_layout()
plt.savefig(f"fig/example_gp_model_sampling.pdf")
plt.close()





In [50]:
import numpy as np
import matplotlib.pyplot as plt

def rbf_kernel(X1, X2, length_scale, signal_variance):
    """
    Radial Basis Function (RBF) Kernel.
    Args:
        X1: (n_samples_1, n_features) numpy array.
        X2: (n_samples_2, n_features) numpy array.
        length_scale: Length scale (\ell).
        signal_variance: Signal variance (\sigma_f^2).
    Returns:
        Covariance matrix of shape (n_samples_1, n_samples_2).
    """
    sqdist = np.sum(X1**2, axis=1).reshape(-1, 1) + np.sum(X2**2, axis=1) - 2 * np.dot(X1, X2.T)
    return signal_variance * np.exp(-0.5 * sqdist / length_scale**2)

def gaussian_process_predict(X_train, y_train, X_test, length_scale, signal_variance, noise_variance):
    """
    Gaussian Process prediction from scratch.
    Args:
        X_train: Training inputs (n_train, n_features).
        y_train: Training targets (n_train, 1).
        X_test: Test inputs (n_test, n_features).
        length_scale: Length scale hyperparameter.
        signal_variance: Signal variance hyperparameter.
        noise_variance: Noise variance (\sigma_n^2).
    Returns:
        mean: Predictive mean (n_test, 1).
        cov: Predictive covariance (n_test, n_test).
    """
    # Compute kernels
    K = rbf_kernel(X_train, X_train, length_scale, signal_variance) + noise_variance * np.eye(len(X_train))
    K_s = rbf_kernel(X_train, X_test, length_scale, signal_variance)
    K_ss = rbf_kernel(X_test, X_test, length_scale, signal_variance)

    # Compute the mean and covariance of the predictive distribution
    K_inv = np.linalg.inv(K)
    mean = K_s.T @ K_inv @ y_train
    cov = K_ss - K_s.T @ K_inv @ K_s

    return mean.ravel(), cov

def sample_from_gp(mean, cov, n_samples=1):
    """
    Sample functions from a Gaussian Process.
    Args:
        mean: Predictive mean (n_test,).
        cov: Predictive covariance (n_test, n_test).
        n_samples: Number of samples to draw.
    Returns:
        Samples of shape (n_samples, n_test).
    """
    return np.random.multivariate_normal(mean, cov, size=n_samples)

# Generate training data
np.random.seed(7)
X_train = np.random.randn(20, 1) * 15
y_train = 0.1 * X_train + np.sin(X_train) + np.random.randn(20, 1) * 0.01

# Define test points
X_test = np.linspace(-50, 40, 2000).reshape(-1, 1)

# Define kernel hyperparameters
length_scale = 3.0
signal_variance = 2.0
noise_variance = 0.1

# Perform GP prediction
mean, cov = gaussian_process_predict(X_train, y_train, X_test, length_scale, signal_variance, noise_variance)

# Sample functions from the GP posterior
n_samples = 4
y_samples = sample_from_gp(mean, cov, n_samples)

# Plot results
plt.plot(X_train, y_train, "b.", label="Training data")
plt.title(f"GP Samples with \u2113={length_scale}, $\u03c3_f^2$={signal_variance}, $\u03c3_n^2$={noise_variance}")
plt.plot(X_test, y_samples[0], "g-", alpha=0.5, label="GP samples")
for i in range(1, n_samples):
    plt.plot(X_test, y_samples[i], "g-", alpha=0.5)
plt.fill_between(X_test.ravel(), mean - 1.96 * np.sqrt(np.diag(cov)), mean + 1.96 * np.sqrt(np.diag(cov)), alpha=0.1, color="red")
plt.xlabel("x")
plt.ylabel("y", rotation=0)
plt.legend()
plt.axis([-50, 40, -4, 6])
plt.tight_layout()
plt.savefig("fig/example_gp_model_6_sampling.pdf")
plt.close()
